# Modelo - Treino e Teste

**Importações**

In [21]:
# Bibliotecas Necessárias para o modelo de recomendação
import pandas as pd
pd.set_option('display.max_columns', None)
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV 
from surprise import accuracy
from collections import defaultdict

In [22]:
# Carregamento das bases (o link dos arquivos estão no READ.md)
olist_customers = pd.read_csv("olist_customers_dataset.csv", sep=",")
olist_orders = pd.read_csv("olist_orders_dataset.csv", sep=",")
olist_order_payments = pd.read_csv("olist_order_payments_dataset.csv", sep=",")
olist_order_reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep=",")
olist_order_items = pd.read_csv("olist_order_items_dataset.csv", sep=",")

In [23]:
# Juntando todas as bases
dataframe = olist_customers.merge(olist_orders, on="customer_id") \
                           .merge(olist_order_payments, on="order_id") \
                           .merge(olist_order_reviews, on="order_id") \
                           .merge(olist_order_items, on="order_id")

<span style="font-size:24px">**Filtro de Dados Utilizados no Modelo de Recomendação**</span>

Esse momento, optei por filtrar do DataFrame original apenas os pedidos que foram entregues para os consumidores. Essa decisão é pelo entendimento que outros status como **não entregues, a caminho ou pgto efutuado** possuiam notas e poderiam atrapalhar o modelo de recomendação de produtos que foram recebido e devidametne avaliados.

Também nessa seleção, precisamos apenas das colunas do ID dos usuários, ID do produto e a avaliação das ao produto.

In [24]:
df_recom = dataframe[dataframe['order_status'] == 'delivered'][['customer_unique_id', 'product_id', 'review_score']]
df_recom.head()

,customer_unique_id,product_id,review_score
0,861eff4711a542e4b93843c6dd7febb0,a9516a079e37a9c9c36b9b78b10169e8,4
1,290c77bc529b7ac935b93aa66c333dc3,4aa6014eceb682077f9dc4bffebc05b0,5
2,060e732b5b29e8181a18229c7b0b2b5e,bd07b66896d6f1494f5b86251848ced7,5
3,259dac757896d24d7702b9acbbff3f3c,a5647c44af977b148e0a3a4751a09e2e,5
4,345ecd01c38d18a9036ed96c73b8d066,9391a573abe00141c56e38d84d7d5b3b,5


In [25]:
# Verificando se há valores vazios
df_recom.isna().value_counts()

customer_unique_id  product_id  review_score
False               False       False           114859
Name: count, dtype: int64

In [26]:
# Renomeando o nome das colunas
df_recom.rename(columns={
    'customer_unique_id':'user_id',
    'product_id':'product_id',
    'review_score':'rating'
}, inplace=True)

In [ ]:
df_recom['user_mean'] = df_recom.groupby('user_id')['rating'].transform('mean')
df_recom['user_reviews_count'] = df_recom.groupby('user_id')['rating'].transform('count')
df_recom['normalized_rating'] = df_recom['rating'] - df_recom['user_mean']

In [77]:
df_recom_reviw_1 = df_recom[df_recom['user_reviews_count'] > 1]

In [74]:
avaliacoes_por_user_id = df_recom['user_id'].value_counts()
usuarios_por_qtd_avaliacoes = avaliacoes_por_user_id.value_counts().sort_index()
usuarios_por_qtd_avaliacoes_df = pd.DataFrame({
    'qtd_avaliacoes': usuarios_por_qtd_avaliacoes.index,
    'qtd_usuarios': usuarios_por_qtd_avaliacoes.values
})
usuarios_por_qtd_avaliacoes_df.columns = ['qtd_avaliacoes', 'qtd_usuarios']
usuarios_por_qtd_avaliacoes_df

,qtd_avaliacoes,qtd_usuarios
0,1,78803
1,2,10170
2,3,1970
3,4,944
4,5,328
5,6,289
6,7,70
7,8,46
8,9,24
9,10,25


In [78]:
# Fazendo a leitura e carregamento dos dados para o formato da lib Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_recom[['user_id', 'product_id', 'rating']], reader)
data_normalized = Dataset.load_from_df(df_recom[['user_id', 'product_id', 'normalized_rating']], reader)
data_normalized_1 = Dataset.load_from_df(df_recom_reviw_1[['user_id', 'product_id', 'normalized_rating']], reader)

<span style="font-size:24px">**Configurando Melhores Carecterísticas do Modelo**</span>

A biblioteca Surprise tem uma função bastante interessante nos deixa escolher alguns hiper parâmetros que nos auxiliam a encontrar a melhor configuração para o modelo de configuração através da função **GridSearchCV**. Os hiper parâmetros que podem ser definidos:

<span style="font-size:20px">**n_factors**</span>

Trata sobre a consideração de carecrísticas não vísiveis dos usuários que não estão no modelo.
- Entre 10 à 30 características ocultas é aplicado para modelos mais simples e diminuindo a chance de overfitting do modelo.
- Acima de 100 é aplicado para modelo mais complexos e pode aumentar a possibildiade overfitting para bases de dados pequenas.

<span style="font-size:20px">**lr_all**</span>

Taxa de aprendizado do modelo, controla o tamanho dos passos que o modelo segue durante o treino. Esse parâmetro é o que "ajusta" os pesos a cada iteração.
- Aplicando um valor baixo, como 0.002, o treino do modelo é mais lento, porém é considerado mais estável.
- Aplicando um valor alto, como 0.01, o treino do modelo é mais rápido, porém pode oscilar.

<span style="font-size:20px">**reg_all**</span>

Esse parâmetro é o que "penaliza", em maior ou menor peso, a previsão de dados mais distoantes. Esse parâmetro auxília para evitar o overfitting do modelo.
- Aplicando um valor baixo, como 0.02, há pouca penalização, aumento a possibilidade de overfitting.
- Aplicando um valor alto, como 0.01, há mais penalização, podendo substimar relações reais.

# Apenas usuários com pelo menos 5 avaliações?
# Qtd Média de avaliação por usuário
# Normalizar notas por usuário?

Modelos como SVD, SVD++, e BaselineOnly funcionam melhor quando os dados foram centrados em torno de médias.

O SVD, por exemplo, decompõe a matriz de interações — se as notas estiverem normalizadas, a decomposição converge melhor e capta padrões reais.

In [17]:
param_grid = {
    'n_factors': [50, 100, 125, 150],
    'lr_all': [0.005, 0.008, 0.009, 0.01, 0.02],
    'reg_all': [0.05, 0.075, 0.1, 0.15, 0.2]
}
gs = GridSearchCV(SVD, #Modelo
                  param_grid, #Hiper parâmetros
                  measures=['rmse', 'mae'], #Métricas de Avaliação
                  cv=5, #Qtd de recortes na bases de dados 
                  joblib_verbose=2) # Controle do nível de verbosidade
gs.fit(data) #Treino para encontrar os melhos parâmetros

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   50.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  3.6min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  9.7min


In [18]:
print('RMSE', gs.best_params['rmse'])
print(f"RMSE Score: {gs.best_score['rmse']:.4f}")
print('\nMAE', gs.best_params['mae'])
print(f"MAE Score: {gs.best_score['mae']:.4f}")

RMSE {'n_factors': 150, 'lr_all': 0.02, 'reg_all': 0.075}
RMSE Score: 1.1370

MAE {'n_factors': 150, 'lr_all': 0.02, 'reg_all': 0.05}
MAE Score: 0.8262


In [ ]:
param_grid = {
    'n_factors': [1, 2, 5, 10, 50, 100],
    'lr_all': [0.0005, 0.001, 0.0015, 0.002, 0.005, 0.01],
    'reg_all': [0.01, 0.015, 0.02, 0.025, 0.05, 0.1]
}
gs = GridSearchCV(SVD, #Modelo
                  param_grid, #Hiper parâmetros
                  measures=['rmse', 'mae'], #Métricas de Avaliação
                  cv=5, #Qtd de recortes na bases de dados 
                  joblib_verbose=2) # Controle do nível de verbosidade
gs.fit(data_normalized) #Treino para encontrar os melhos parâmetros

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   29.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  2.0min
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  4.5min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  8.2min
[Parallel(n_jobs=1)]: Done 1012 tasks      | elapsed: 16.1min


In [42]:
print('RMSE', gs.best_params['rmse'])
print(f"RMSE Score: {gs.best_score['rmse']:.4f}")
print('\nMAE', gs.best_params['mae'])
print(f"MAE Score: {gs.best_score['mae']:.4f}")

RMSE {'n_factors': 1, 'lr_all': 0.0005, 'reg_all': 0.01}
RMSE Score: 1.0175

MAE {'n_factors': 100, 'lr_all': 0.005, 'reg_all': 0.01}
MAE Score: 1.0065


In [79]:
param_grid = {
    'n_factors': [1, 2, 5, 10, 50, 100],
    'lr_all': [0.0005, 0.001, 0.0015, 0.002, 0.005, 0.01],
    'reg_all': [0.01, 0.015, 0.02, 0.025, 0.05, 0.1]
}
gs = GridSearchCV(SVD, #Modelo
                  param_grid, #Hiper parâmetros
                  measures=['rmse', 'mae'], #Métricas de Avaliação
                  cv=5, #Qtd de recortes na bases de dados 
                  joblib_verbose=2) # Controle do nível de verbosidade
gs.fit(data_normalized_1) #Treino para encontrar os melhos parâmetros

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    9.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   37.9s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:  1.4min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  2.6min
[Parallel(n_jobs=1)]: Done 1012 tasks      | elapsed:  4.8min


In [80]:
print('RMSE', gs.best_params['rmse'])
print(f"RMSE Score: {gs.best_score['rmse']:.4f}")
print('\nMAE', gs.best_params['mae'])
print(f"MAE Score: {gs.best_score['mae']:.4f}")

RMSE {'n_factors': 1, 'lr_all': 0.0005, 'reg_all': 0.01}
RMSE Score: 1.0548

MAE {'n_factors': 100, 'lr_all': 0.005, 'reg_all': 0.01}
MAE Score: 1.0208


Após realizar o treino com vários valores para parâmetros, chegou o momento de verificar, com base no resultado do RMSE e o MAE, quais valores dos hiper parâmetros são os mais ideais para gerar um modelo de recomendação de maior qualidade de previsão.

Antes de verificar os melhores valores para os hiper parâmetros, vamos relembrar o que significa o RMSE e o MAE que vamos usar como base para essa escolha:

<span style="font-size:20px">**Root Mean Squared Error (RMSE)**</span>

Essa métrica mede o erro médio ao quadrado entre o valor previsto e o valor real, auxilia no entendimento de quando o modelo pode cometer erros mais altos de previsão.

<span style="font-size:20px">**Mean Absolute Error (MAE)**</span>

Essa métrica mede a diferença absoluta média entre o valor previsto e o valor real, auxilia no entendimento da qualidade do modelo em prever os valores reais.


**Interpretação**

Para esse caso, que temos notas de produtos entre 1 e 5, poderiamos interpretar o RMSE e o MAE da seguinte forma:

Abaixo 0.7 - Excelente | Entre 0.7 e 0.9 - Bom | Entre 0.9 e 1.1 - Ok | Acima de 1.1 - Ruim

**Treino, Teste e Validação**

In [11]:
# Divisão dos dados
trainset, testset = train_test_split(data, test_size=0.25)

# Modelo com os melhores parâmetros
best_model = gs.best_estimator['rmse']
best_model.fit(trainset)

# Avaliação no conjunto de teste
predictions = best_model.test(testset)
print("RMSE no teste:", accuracy.rmse(predictions))
print("MAE no teste:", accuracy.mae(predictions))

RMSE: 1.1667
RMSE no teste: 1.1667449742755942
MAE:  0.8811
MAE no teste: 0.8810697263300789


In [12]:
def precision_recall_f1_at_k(predictions, k=3, threshold=4.0):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        
    precisions, recalls, f1s = [], [], []
    for est_true in user_est_true.values():
        est_true.sort(key=lambda x: x[0], reverse=True)
        top_k = est_true[:k]
        tp = sum((true_r >= threshold) for _, true_r in top_k)
        fp = k - tp
        fn = sum((true_r >= threshold) for _, true_r in est_true[k:])
        
        precision = tp / (tp + fp) if (tp + fp) else 0
        recall = tp / (tp + fn) if (tp + fn) else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0
        
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    
    return {
        'Precision': sum(precisions) / len(precisions),
        'Recall': sum(recalls) / len(recalls),
        'F1': sum(f1s) / len(f1s)
    }


In [13]:
metrics = precision_recall_f1_at_k(predictions, k=3)
print("Métricas Top-N:", metrics)

Métricas Top-N: {'Precision': 0.2741800566811954, 'Recall': 0.7766103223850972, 'F1': 0.4011694343335997}


**Treino com todos os dados**

In [14]:
# Treinar com 100% dos dados
trainset_full = data.build_full_trainset()
best_model.fit(trainset_full)

In [15]:
# Treinar com 100% dos dados
trainset_full = data.build_full_trainset()
best_model.fit(trainset_full)

# Recomendação para usuário 1: itens que ele ainda não avaliou
user_id = "345ecd01c38d18a9036ed96c73b8d066"
all_items = set(iid for (_, iid, _) in trainset_full.all_ratings())
rated_items = set(j for (j, _) in trainset_full.ur[trainset_full.to_inner_uid(user_id)])
unseen_items = all_items - rated_items

# Predizer e ordenar por melhor nota prevista
recommendations = [
    (iid, best_model.predict(user_id, iid).est) for iid in unseen_items
]
recommendations.sort(key=lambda x: x[1], reverse=True)

# Top 3 recomendações
print(f"Top 3 recomendações para o usuário {user_id}:")
for iid, score in recommendations[:3]:
    print(f"Item {iid} - Nota prevista: {score:.2f}")


Top 3 recomendações para o usuário 345ecd01c38d18a9036ed96c73b8d066:
Item 0 - Nota prevista: 4.23
Item 1 - Nota prevista: 4.23
Item 2 - Nota prevista: 4.23


In [16]:
# Função para recomendar produtos a um usuário específico
def recomendar_produtos(user_id, df, model, n_recommendations=3):
    """Gera recomendações de produtos para um usuário."""
    
    # Obtém todos os produtos únicos
    all_products = df['product_id'].unique()
    
    # Obtém produtos já avaliados pelo usuário
    rated_products = df[df['user_id'] == user_id]['product_id'].values
    
    # Filtra apenas os produtos que o usuário ainda não avaliou
    products_to_predict = [p for p in all_products if p not in rated_products]
    
    # Faz previsões para esses produtos
    predictions = [(p, model.predict(user_id, p).est) for p in products_to_predict]
    
    # Ordena por nota prevista (maior para menor)
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    return predictions[:n_recommendations]

# Exemplo: Recomendação para o usuário 1
user_id = 1
recommendations = recomendar_produtos(user_id, df, model)

print(f"\nRecomendações para o usuário {user_id}:")
for product, rating in recommendations:
    print(f"Produto {product} - Nota prevista: {rating:.2f}")

NameError: name 'df' is not defined

# Recomendação de Produtos

**Separação dos dados para recomendação**

In [ ]:
df_ultima_compra = dataframe.groupby("customer_unique_id").agg({'order_purchase_timestamp':max})
df_ultima_compra.reset_index(inplace=True)

C:\Users\Computador\AppData\Local\Temp\ipykernel_4784\2487374714.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ultima_compra = dataframe.groupby("customer_unique_id").agg({'order_purchase_timestamp':max})


In [ ]:
df_ultima_compra['order_purchase_timestamp'] = pd.to_datetime(df_ultima_compra['order_purchase_timestamp'])
data_referencia = pd.to_datetime('2018-09-10')
df_ultima_compra['dias_desde_da_ultima_compra'] = (data_referencia - df_ultima_compra['order_purchase_timestamp']).dt.days

In [ ]:
df_30_90_dias = df_ultima_compra[(df_ultima_compra['dias_desde_da_ultima_compra'] >= 30) & (df_ultima_compra['dias_desde_da_ultima_compra'] <= 90)]['customer_unique_id']
df_30_90_dias = df_30_90_dias.to_list()